# Non-Motor Assessments
In this notebook we will analyze some aspects of the non-motor variables, measurements done in PPMI dataset

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# All non-motor test sheets (what is there in each one), is described on page 8 of the curation_summary document
# In this notebook we will look at each file in the full dataset related to non-motor tests

In [ ]:
path = ### PUT PATH TO DATA HERE ###

### Benton Judgement of Line Orientation
The Benton Judegment of Line Orientation test measures a person's ability to match the angle and orientation of lines in space. Subjects are asked to match two angled lines to a set of 11 lines that are arranged in a semicircle and separated 18 degrees from each other. The complete test has 30 items. 
That is why we have 30 columns for BJLOT<num> where num=1 to 30
But certain columns only occur on certain visits
It seems like on certain days only a few lines in the orientation were tested on. And on other days some other lines.

There is `JLO_TOTRAW` variable which is the sum of the scores (1 for correct and 0 for incorrect) on a particular visit.
On taking a closer look. Some visits do odd number of lines, some do even. Which means effectively everytime only half of them are done. We will look at the stats related to this.
The details on which visit has odd items and which has even read from Pg 67 in the PPMI_Protocol document.

`JLO_TOTCALC` scales the raw score and gives how well a person did out of 30 - which is the standard for the test. This will be an important measure for our purposes.

`DVS_JLO_MSSA` gives `JLO_TOTRAW` + `COMM` but not sure when `COMM` appears. It is rare in the CSV file.

`DVS_JLO_MSSAE` seems some kind of error or a floating point representation of the `DVS_JLO_MSSA` column. Didn't find anything in the document related to this yet.

In [ ]:
file = os.path.join(path, "Benton_Judgment_of_Line_Orientation.csv")
df_1 = pd.read_csv(file)
print("Total number of rows are:",len(df_1))
df_1.head(3)

In [ ]:
print("Number of total records (rows):",len(df_1))
# Looks like a few patients did not do the BL visit
print("Number of records for baseline visit:",len(df_1[df_1.EVENT_ID=="BL"]))
print("Unique patients:", len(df_1.PATNO.unique()))

In [ ]:
# Plot to show the number of visits done by the patients - Like how many people did 7 visits, 8 visits and so on
# We do this by grouping PATNO
df_1.groupby(['PATNO']).PATNO.count()
plt.figure(figsize=(13,6))
plt.hist(df_1.groupby(['PATNO']).PATNO.count(),bins=range(0,11))
plt.xlabel('Number of visits')
plt.ylabel('Number of subjects')

# So about 250 subjects have done 6 visits (these are across months). These patients can be monitored for our purposes.

In [ ]:
events_series = df_1.groupby(['EVENT_ID']).EVENT_ID.count()


events_series.plot.bar()

In [ ]:
# Distribution of age
plt.figure(figsize=(13,6))
plt.hist(df_1['AGE_ASSESS_JLO'].dropna())

In [ ]:
# Check the distribution of scores for TOTCALC
plt.figure(figsize=(13,6))
plt.hist(df_1['JLO_TOTCALC'].dropna().astype(int), bins=np.arange(0,32)-0.5)

In [ ]:
# Add graphs to show which visits have which benton test done. But it is also in the PDF so we will do this later

In [ ]:
# There are reasons given for some of them and for some the other columns have indicators but still the total is 
# missing. Can discard these entries.
df_1[df_1['JLO_TOTCALC'].isnull()].PATNO.count()

### Epworth Sleepiness Scale
The questionnaire asks the subject to rate his or her probability of falling asleep on a scale of increasing probability from 0 to 3 for eight different situations that most people engage in during their daily lives, though not necessarily every day

Can look at non-motor03 in the curated dataset to see the activities for each column. 3 is more sleepy, 1 is less sleepy and so on. Its a scale of 0-3.

`PTCGBOTH` variable might be for Patient-Care giver-Both (1-2-3 values respectively). The curated one has a column that asks who reported the result. So this most likely means the same. Doesn't seem to be that important for our purposes.

There isn't any aggregation across the sleepiness scores. They are all individual depending on certain times in the day and the activity being done by the patient. From overall analysis all of these columns (prefixed with ESS) would be important for prediction.

In [ ]:
file = os.path.join(path, "Epworth_Sleepiness_Scale.csv")
df_2 = pd.read_csv(file)
print("Total number of rows are:",len(df_2))
df_2.head(3)

In [ ]:
# We'll do graphs similar to the ones for Benton
# Plot to show the number of visits done by the patients - Like how many people did 7 visits, 8 visits and so on
# We do this by grouping PATNO
df_2.groupby(['PATNO']).PATNO.count()
plt.figure(figsize=(13,6))
plt.hist(df_2.groupby(['PATNO']).PATNO.count(), bins=range(0,13))
plt.xlabel('Number of visits')
plt.ylabel('Number of subjects')

# Here number of visits done are pretty consistent. That means we would need to consider more patients, but some
# of them did certain visits, others did some others. Alignment will be an issue

In [ ]:
events_series_2 = df_2.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_2)

events_series_2.plot.bar()

### Geriatric Depression Scale (GDS-15)
The Geriatric Depression Scale (GDS) is a 30-item self-report assessment used to identify depression in the elderly. We only take 15 of these. Can look at their full form in non-motor04 file from curated dataset.

They are all yes/no type of questions. 1 means yes and 0 means no. All the GDS columns would prove useful as a feature. For this and ESS, a single concatenated encoding vector could be used.

There is no aggregate score available in the full dataset file. `F_STATUS` appears with only 4 rows having 'S' and rest all having 'V', not sure what it stands for.

In [ ]:
file = os.path.join(path, "Geriatric_Depression_Scale__Short_.csv")
df_3 = pd.read_csv(file)
print("Total number of rows are:",len(df_3))
df_3.head(3)

In [ ]:
# We'll do similar graphs for this too due to the similarity in structure in all these tables
# Plot to show the number of visits done by the patients - Like how many people did 7 visits, 8 visits and so on
# We do this by grouping PATNO
df_3.groupby(['PATNO']).PATNO.count()
plt.figure(figsize=(13,6))
plt.hist(df_3.groupby(['PATNO']).PATNO.count(), bins=np.arange(0,12)-0.5)
plt.xlabel('Number of visits')
plt.ylabel('Number of subjects')

# Again there seems to be an even spread but falls off after 8, so we can skip those patients or not take
# their final few visits into account

In [ ]:
events_series_3 = df_3.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_3)

events_series_3.plot.bar()

# Like ESS seems to fall off after the 4th visit

### Hopkins Verbal Learning Test – Revised
HVLT - A new test of verbal learning and memory, the Hopkins Verbal Learning Test, was developed. The test consists of three trials of free-recall of a 12-item, semantically categorized list, followed by yes/no recognition. The revised might have some additional things.

The full forms for our column headers can be found in curated non-motor05 file. All the DVT columns in the full dataset are the dervied values and hence are important features for the model. `HVLTRSN` is the booklet version used for the test.

It also looks like a higher value for all these tests is better. Which can give a high score overall. We should probably look to use the derived scores as features, since those encode the most important part of the information.

In [ ]:
file = os.path.join(path, "Hopkins_Verbal_Learning_Test.csv")
df_4 = pd.read_csv(file)
print("Total number of rows are:",len(df_4))
df_4.head(3)

In [ ]:
# Let's see the different versions and which one was used the most

# df_4.HVLTVRSN.min()

plt.figure(figsize=(10,5))
plt.hist(df_4.HVLTVRSN.dropna(), bins=np.arange(0,7)-0.5)
plt.xlabel('Test Booklet Version')
plt.ylabel('Number of instances')

# So we can use just the records for version 1 and 2. Or we can take all since the metrics stay the same

In [ ]:
# Let's look at the distribution of the derived scores across subjects and visits
# df_4.DVT_RECOG_DISC_INDEX.min()

plt.figure(figsize=(13,7))
plt.subplot(2,2,1)
plt.hist(df_4.DVT_TOTAL_RECALL.dropna(), bins=np.arange(20,82)-0.5)
plt.xlabel('Number of instances')
plt.ylabel('DVT_TOTAL_RECALL')

plt.subplot(2,2,2)
plt.hist(df_4.DVT_DELAYED_RECALL.dropna(), bins=np.arange(20,82)-0.5)
plt.xlabel('Number of instances')
plt.ylabel('DVT_DELAYED_RECALL')

plt.subplot(2,2,3)
plt.hist(df_4.DVT_RETENTION.dropna(), bins=np.arange(20,82)-0.5)
plt.xlabel('Number of instances')
plt.ylabel('DVT_RETENTION')

plt.subplot(2,2,4)
plt.hist(df_4.DVT_RECOG_DISC_INDEX.dropna(), bins=np.arange(20,82)-0.5)
plt.xlabel('Number of instances')
plt.ylabel('DVT_RECOG_DISC_INDEX')

# Scores for most of these except the Recall test lies between 50-60; but overall a large chunk between 40 and 60 for
# all of the derived scores

In [ ]:
events_series_4 = df_4.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_4)
# print(events_series.axes)
# print(type(events_series.axes))
# print(events_series.values)

events_series_4.plot.bar()
# Similar to others, gradual drop after a peek at V04

### Letter Number Sequencing
This is another neuropsychological test done on certain visits. Participants must recall a series of numbers in increasing order and letters in alphabetical order. 1 is correct and 0 is incorrect for each. There are 7 trials and two parts to each. 

`LNS_TOTRAW`(LNS-Sum Questions 1-7) and `DVS_LNS` (Derived-LNS Scaled Score) seem to be the important metrics we should add to the feature test from this test. Or only the derived one should also work. Not sure what calculation goes into the derived score, at some points values have increased and at some decreased from the TOTRAW

In [ ]:
file = os.path.join(path, "Letter_-_Number_Sequencing__PD_.csv")
df_5 = pd.read_csv(file)
print("Total number of rows are:",len(df_5))
df_5.head(3)

In [ ]:
# Let's look at the distribution of the derived scores across subjects and visits
# df_5.DVS_LNS.max()

plt.figure(figsize=(13,7))
plt.subplot(2,1,1)
plt.hist(df_5.LNS_TOTRAW.dropna(), bins=np.arange(0,22)-0.5)
plt.xlabel('Number of instances')
plt.ylabel('LNS_TOTRAW')

plt.subplot(2,1,2)
plt.hist(df_5.DVS_LNS.dropna(), bins=np.arange(0,22)-0.5)
plt.xlabel('Number of instances')
plt.ylabel('DVS_LNS')

# Look like Gaussians which is expected on any test

In [ ]:
events_series_5 = df_5.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_5)
# print(events_series.axes)
# print(type(events_series.axes))
# print(events_series.values)

events_series_5.plot.bar()
# Again we see the same trend. Seems to be consistent across all the non-motor tables

### Montreal Cognitive Assessment (MoCA)
The Montreal Cognitive Assessment (MoCA) is a widely used screening assessment for detecting cognitive impairment. The details and also the column headers can be seen in curated non-motor07 file. 27 different tests are given to the subjects. The specific tests are listed in the curated file.

`MCATOT` is the one that is the epitome of this sheet. Should add to the feature set. `F_STATUS` is there in this file too but still not sure what it stands for.

In [ ]:
file = os.path.join(path, "Montreal_Cognitive_Assessment__MoCA_.csv")
df_6 = pd.read_csv(file)
print("Total number of rows are:",len(df_5))
df_6.head(3)

In [ ]:
# Distribution of total scores
# df_6.MCATOT.max()

plt.figure(figsize=(13,7))
plt.hist(df_6.MCATOT.dropna(), bins=np.arange(0,32)-0.5)
plt.xlabel('Number of instances')
plt.ylabel('MCATOT')

# Everyone has a pretty high score on this test

In [ ]:
# Number of patients per visit graph
events_series_6 = df_6.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_6)
# print(events_series.axes)
# print(type(events_series.axes))
# print(events_series.values)

events_series_6.plot.bar()

### Olfactory Testing (UPSIT)
The University of Pennsylvania Smell Identification Test (UPSIT) is a test that is commercially available for smell identification to test the function of an individual's olfactory system.

Response columns have the thing they were actually told to smell and then whether they got it correct or not is in another column. Only the `TOTAL_CORRECT` seems to have some kind of signal related to this test. The percentage column I'm not sure about. It seems off or don't seem to get the calculation they have used there.

Surprisingly this file doesn't have which visit happened on which visit. We would have to align using the date.

`non-motor15` file has this test

In [ ]:
file = os.path.join(path, "Olfactory_UPSIT.csv")
df_7 = pd.read_csv(file)
print("Total number of rows are:",len(df_7))
df_7.head(3)

In [ ]:
# Distribution of the total scores for this test
# df_7.TOTAL_CORRECT.max()

plt.figure(figsize=(13,7))
plt.hist(df_7.TOTAL_CORRECT.dropna(), bins=np.arange(2,42)-0.5)
plt.xlabel('TOTAL_CORRECT')
plt.ylabel('Number of instances')

# Tending to the higher side for most patients

In [ ]:
# Number of patients per visit graph
events_series_7 = df_7.groupby(['COMPLT_DATE']).COMPLT_DATE.count()
print(events_series_7)
# print(events_series.axes)
# print(type(events_series.axes))
# print(events_series.values)

events_series_7.plot.bar()

# The problem with doing dates is that it is too spread out. So we might need to map these to visits like the other
# tests

### QUIP
This is a questionnaire about gambling, buying, etc. It is a nerobehavioral questionnaire.

1 means yes and 0 means no for answers to the questions. `non-motor08` (curated dataset) has each question listed. In the curated one there is a summary score, but in the full dataset there isn't. So we might need to encode an entire vector so each question holds value on its own as a feature.

In [ ]:
file = os.path.join(path, "QUIP_Current_Short.csv")
df_8 = pd.read_csv(file)
print("Total number of rows are:",len(df_8))
df_8.head(3)

In [ ]:
# Picked five questions randomly to see the values
plt.figure(figsize=(13,7))
plt.subplot(2,2,1)
plt.hist(df_8.TMGAMBLE.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('TMGAMBLE')
plt.ylabel('Number of instances')

plt.subplot(2,2,2)
plt.hist(df_8.CNTRLBUY.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('CNTRLBUY')
plt.ylabel('Number of instances')

plt.subplot(2,2,3)
plt.hist(df_8.TMEAT.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('TMEAT')
plt.ylabel('Number of instances')

plt.subplot(2,2,4)
plt.hist(df_8.TMTORACT.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('CNTRLDSM')
plt.ylabel('Number of instances')

# Most of them have a high number of No replies. Can't tell how much information we will get from this test

In [ ]:
# Number of patients per visit graph
events_series_8 = df_8.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_8)

events_series_8.plot.bar()
# Again the tapering trend down from V04

### REM Sleep Behavior Questionnaire
This is a Sleep Disorder Test. Very similar format to the QUIPCS in the previous section. This one has questions related to sleeping problems. Full forms for column headers in `non-motor09` file.

0 stands of No. 1 for yes. `CNSOTHCM` column specifies any other disorders. No summary score in the full dataset so may be we have to do our aggregation or use the columns individually as features.

In [ ]:
file = os.path.join(path, "REM_Sleep_Disorder_Questionnaire.csv")
df_9 = pd.read_csv(file)
print("Total number of rows are:",len(df_9))
df_9.head(3)

In [ ]:
# Picked five questions randomly to see the values
plt.figure(figsize=(13,7))
plt.subplot(2,2,1)
plt.hist(df_9.DRMVIVID.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('DRMVIVID')
plt.ylabel('Number of instances')

plt.subplot(2,2,2)
plt.hist(df_9.DRMAGRAC.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('DRMAGRAC')
plt.ylabel('Number of instances')

plt.subplot(2,2,3)
plt.hist(df_9.SLPLMBMV.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('SLPLMBMV')
plt.ylabel('Number of instances')

plt.subplot(2,2,4)
plt.hist(df_9.MVAWAKEN.dropna(), bins=np.arange(0,3)-0.5)
plt.xlabel('MVAWAKEN')
plt.ylabel('Number of instances')

# This one has a better split between 0s and 1s

In [ ]:
# Number of patients per visit graph
events_series_9 = df_9.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_9)
# print(events_series.axes)
# print(type(events_series.axes))
# print(events_series.values)

events_series_9.plot.bar()
# This one goes down from V06

### SCOPA-AUT
This is a autonomic test. It is for gauging the problems the patient is facing with respect to bodily functions. The questionnaire is answered with a scale 0-3 (0-Never, 1-Sometimes, 2-Regularly, 3-Often). `non-motor10` maps to this one.

`SCAU23AT`, mentions any kind of drugs that might be taken by the patient. Similar thing for all columns suffixed with `AT`, `BT`, `CT` (so the columns ending it `T`). In each case it asks for medicines taken for the previous column. This may or may not be useful to us with the initial analysis. Might be useful later. Some are yes/no questions. And then they have to give the specific answer in the next column as mentioned above.

We can again aggregate (sum) these scores. Lower score means better.

In [ ]:
file = os.path.join(path, "SCOPA-AUT.csv")
df_10 = pd.read_csv(file)
print("Total number of rows are:",len(df_10))
df_10.head(3)

In [ ]:
# Graphs for some of the responses ignoring the one that were not answered by some patients
plt.figure(figsize=(13,7))
plt.subplot(2,2,1)
plt.hist(df_10.SCAU2.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('SCAU2')
plt.ylabel('Number of instances')

plt.subplot(2,2,2)
plt.hist(df_10.SCAU4.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('SCAU4')
plt.ylabel('Number of instances')

plt.subplot(2,2,3)
plt.hist(df_10.SCAU9.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('SCAU9')
plt.ylabel('Number of instances')

plt.subplot(2,2,4)
plt.hist(df_10.SCAU23.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('SCAU23')
plt.ylabel('Number of instances')

In [ ]:
# Number of patients per visit graph
events_series_10 = df_10.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_10)

events_series_10.plot.bar()

### Semantic Fluency
Neuropsychological Test. There are two derived scores for animal fluency. One is the T score and one is the scaled score. There are also total number of vegetables and fruits that were shown to participants. Can take these as features.

The `T score` looks important. Details in `non-motor11` file.

In [ ]:
file = os.path.join(path, "Semantic_Fluency.csv")
df_11 = pd.read_csv(file)
print("Total number of rows are:",len(df_11))
df_11.head(3)

In [ ]:
# print(df_11.VLTFRUIT.dropna().min(),df_11.VLTFRUIT.dropna().max())

# Graphs for the total scores
plt.figure(figsize=(13,7))
plt.subplot(2,2,1)
plt.hist(df_11.VLTANIM.dropna(), bins=np.arange(0,43)-0.5)
plt.xlabel('VLTANIM')
plt.ylabel('Number of instances')

plt.subplot(2,2,2)
plt.hist(df_11.VLTVEG.dropna(), bins=np.arange(0,43)-0.5)
plt.xlabel('VLTVEG')
plt.ylabel('Number of instances')

plt.subplot(2,2,3)
plt.hist(df_11.VLTFRUIT.dropna(), bins=np.arange(0,43)-0.5)
plt.xlabel('VLTFRUIT')
plt.ylabel('Number of instances')

In [ ]:
# Number of patients per visit graph
events_series_11 = df_11.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_11)

events_series_11.plot.bar()
# Not many responses from the few initial visits

### State-Trait Anxiety Inventory for Adults
Neurobehavioral Test. The State-Trait Anxiety Inventory (STAI) is a psychological inventory based on a 4-point Likert scale and consists of 40 questions on a self-report basis.

As a general observation, the curated dataset sums up values in a lot of sheets. We can also resort to that kind of aggregation. This sheet contains the 40 questions and their results. The questions are in the file `non-motor12`. Questions like I feel calm, I feel tensed. So we can't really sum the scores up because there are negative and positive questions. (1 is very less and 3 is very much).

In [ ]:
file = os.path.join(path, "State-Trait_Anxiety_Inventory.csv")
df_12 = pd.read_csv(file)
print("Total number of rows are:",len(df_12))
df_12.head(3)

In [ ]:
# Distribution of scores for 4 randomly picked questions
# First two are positive and the next two negative
plt.figure(figsize=(13,7))
plt.subplot(2,2,1)
plt.hist(df_12.STAIAD5.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('STAIAD5')
plt.ylabel('Number of instances')

plt.subplot(2,2,2)
plt.hist(df_12.STAIAD11.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('STAIAD11')
plt.ylabel('Number of instances')

plt.subplot(2,2,3)
plt.hist(df_12.STAIAD17.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('STAIAD17')
plt.ylabel('Number of instances')

plt.subplot(2,2,4)
plt.hist(df_12.STAIAD36.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('STAIAD36')
plt.ylabel('Number of instances')

In [ ]:
# Number of patients per visit graph
events_series_12 = df_12.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_12)


events_series_12.plot.bar()

### Symbol Digit Modalities
Neuropsychological Test. Similar to semantic fluency, there is a derived scaled score and T score. The derived scores seem to be important. Even the total score can be used as a feature

In [ ]:
file = os.path.join(path, "Symbol_Digit_Modalities.csv")
df_13 = pd.read_csv(file)
print("Total number of rows are:",len(df_13))
df_13.head(3)

In [ ]:
# DVSD_SDM - Scaled score
plt.figure(figsize=(10,5))
plt.hist(df_13.DVSD_SDM.dropna())
plt.xlabel('DVSD_SDM')
plt.ylabel('Number of instances')

In [ ]:
# DVT_SDM - T-score
plt.figure(figsize=(10,5))
plt.hist(df_13.DVT_SDM.dropna())
plt.xlabel('DVT_SDM')
plt.ylabel('Number of instances')

In [ ]:
# SDMTOTAL
plt.figure(figsize=(10,5))
plt.hist(df_13.SDMTOTAL.dropna())
plt.xlabel('SDMTOTAL')
plt.ylabel('Number of instances')

In [ ]:
# SDMTVRSN
plt.figure(figsize=(10,5))
plt.hist(df_13.SDMTVRSN.dropna(), bins=np.arange(0,5)-0.5)
plt.xlabel('SDMTVRSN')
plt.ylabel('Number of instances')

In [ ]:
# Number of patients per visit graph
events_series_13 = df_13.groupby(['EVENT_ID']).EVENT_ID.count()
print(events_series_13)

events_series_13.plot.bar()
# Again very few results for the initial screening and such preliminary visits.

## From here we will work on processing each file and writing all the information to a single DF

In [ ]:
# df_13 is sdmt dataframe
df_13.head(3)

In [ ]:
df_13.columns

In [ ]:
df_processed = df_13[['PATNO', 'EVENT_ID', 'INFODT', 'SDMTOTAL']] 
df_processed.head(3)

In [ ]:
df_processed.SDMTOTAL.isnull().sum()

In [ ]:
df_processed[df_processed.SDMTOTAL.isnull()]

In [ ]:
# Processing the feature for df_12 - State Trait Anxiety Index (STAI)

# Guidelines:
# STAIAD1 - STAIAD40.  
# Add values for the following questions:  
# 3, 4, 6, 7, 9, 12, 13, 14, 17, 18, 22, 24, 25, 28, 29, 31, 32, 35, 37, 38, 40.  
# Use reverse scoring for the remaining questions and add to the first score 
# (e.g., if value = 1, add 4 points to score; if value = 2, add 3 points to score, etc).
df_12.head(3)

In [ ]:
# Define a function to get the final STAI
add_stai = [3, 4, 6, 7, 9, 12, 13, 14, 17, 18, 22, 24, 25, 28, 29, 31, 32, 35, 37, 38, 40]
stai_set = set()
for v in add_stai:
    s='STAIAD'+str(v)
    stai_set.add(s)
print(stai_set)

rev_stai = list(range(41))
for i in add_stai:
    rev_stai.remove(i)
print(rev_stai)

stai_rev_set = set()
for v in rev_stai:
    if v==0:
        continue
    s='STAIAD'+str(v)
    stai_rev_set.add(s)
print(stai_rev_set)

In [ ]:
import math
df_12_columns = df_12.columns
def stai_total(row):
    # iterate over the keys
    sum = 0
    for key in df_12_columns:
        if key in stai_set:
            if math.isnan(row[key]):
                return row[key] 
            sum+=int(row[key])
        elif key in stai_rev_set:
            if math.isnan(row[key]):
                return row[key]
            sum+=(5-int(row[key]))
    return sum

In [ ]:
df_12['STAI_TOT'] = df_12.apply(stai_total,axis=1)
df_12.head(3)

In [ ]:
df_12.STAI_TOT.isnull().sum()

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_12[['PATNO','EVENT_ID','INFODT','STAI_TOT']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(3)

In [ ]:
# Processing the feature for df_11 - Semantic Fluency (SFT)

# Guidelines
# Sum of VLTANIM, VLTVEG, VLTFRUIT
df_11.head(3)

In [ ]:
df_11['SFT_TOT'] = df_11[['VLTANIM','VLTVEG','VLTFRUIT']].sum(axis=1)

In [ ]:
df_11.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_11[['PATNO','EVENT_ID','INFODT','SFT_TOT']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(3)

In [ ]:
# Processing the feature for df_10 - SCOPA-AUT

# Guidelines
# SCAU1 - SCAU25.  
# For questions 1-21 (SCAU1 - SCAU21), add 3 points for each response of "9". 
# Otherwise, add the number of points in response.  
# For questions 22-25 (SCAU22 - SCAU25), add 0 points for each respµonse of "9". 
# Otherwise, add the number of points in response.

# Since there are different tests done for male/female. We will take nan=0µb
df_10.head(3)

In [ ]:
add_sa = list(range(1,26))
sa_1 = set()
sa_2 = set()
for v in add_sa:
    s='SCAU'+str(v)
    if v in [22,23,24,25]:
        sa_2.add(s)
    else:
        sa_1.add(s)
print(sa_1)
print(sa_2)

In [ ]:
df_10_columns = df_10.columns
def scopa_total(row):
    # iterate over the keys
    sum = 0
    for key in df_10_columns:
        if key in sa_1:
            if math.isnan(row[key]):
                continue
            if int(row[key])==9:
                sum+=3
            else:
                sum+=int(row[key])
        if key in sa_2:
            if math.isnan(row[key]):
                continue
            if int(row[key])==9:
                sum+=0
            else:
                sum+=int(row[key])
    return sum

In [ ]:
df_10['SCOPA_AUT_TOT'] = df_10.apply(scopa_total,axis=1)
df_10.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_10[['PATNO','EVENT_ID','INFODT','SCOPA_AUT_TOT']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(3)

In [ ]:
# Processing the feature for df_9 - REM Sleep Behavior Questionnaire (REMSLEEP)

# Guidelines
# Add 1 point for each response of "Yes" (1) 
# to any of the following variables:  DRMVIVID, DRMAGRAC, DRMNOCTB, SLPLMBMV, SLPINJUR, 
# DRMVERBL, DRMFIGHT, DRMUMV, DRMOBJFL, MVAWAKEN, DRMREMEM, SLPDSTRB.  
# Add 1 point if any of the following variables has a response of "Yes" (1):  
# STROKE, HETRA, PARKISM, RLS, NARCLPSY, DEPRS, EPILEPSY, BRNINFM, CNSOTH.  
# If any of the previous variables are missing, then RBD score is missing.  
# Subjects with score >=5 are RBD Positive.  Subjects with score <5 are RBD Negative.

# Also checked the curated file - If any of the variables are nan they don't really have a summary score
# So just putting nan for those for now
df_9.head(3)

In [ ]:
rs_set = set(["DRMVIVID","DRMAGRAC","DRMNOCTB","SLPLMBMV","SLPINJUR",
              "DRMVERBL","DRMFIGHT","DRMUMV","DRMOBJFL","MVAWAKEN",
              "DRMREMEM","SLPDSTRB","STROKE","HETRA","PARKISM","RLS",
              "NARCLPSY","DEPRS","EPILEPSY","BRNINFM","CNSOTH"])

In [ ]:
def rem_sleep_total(row):
    # iterate over the keys
    sum = 0
    for key in rs_set:
        if math.isnan(row[key]):
            return row[key] 
        sum+=int(row[key])
    return sum

In [ ]:
df_9['REMSLEEP_TOT'] = df_9.apply(rem_sleep_total,axis=1)
df_9.head(3)

In [ ]:
df_9.REMSLEEP_TOT.isnull().sum()

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_9[['PATNO','EVENT_ID','INFODT','REMSLEEP_TOT']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(3)

In [ ]:
# Processing the feature for df_8 - QUIPCS

# Guidelines
# For Sections A - D, add 1 point 
# if either question has a response of "Yes" (1):  
# Section A:  CNTRLGMB, TMGAMBLE; Section B:  CNTRLSEX, TMSEX; 
# Section C:  CNTRLBUY, TMBUY; Section D:  CNTRLEAT, TMEAT.  
# For Section E, add 1 point for each response of "Yes" (1):  
# TMTORACT, TMTMTACT, TMTRWD.

# A-D: Will be 1 if either or both have Yes. Otherwise 0
# E: We will add across the three questions
# So we will have multiple columns for this feature

# NaN if anything for that particular question is missing

In [ ]:
df_8.head(3)

In [ ]:
# df_8['elderly'] = np.where((df['age']>=50), 1.0, 0.0)
# df_8['QUIP_A'] = df_8.apply(lambda x: 1.0 if (x['CNTRLGMB']==1.0 or x['TMGAMBLE']==1.0) else 0.0, axis=1)
# df_8.head(3)
# Can't use this technique because we won't be able to account for NaNs

In [ ]:
# Need to do all this to handle the NaNs :(
def quip_A(x):
    if math.isnan(x['CNTRLGMB']):
        return x['CNTRLGMB']
    if math.isnan(x['TMGAMBLE']):
        return x['TMGAMBLE']
    if x['CNTRLGMB']==1.0 or x['TMGAMBLE']==1.0:
        return 1.0
    else:
        return 0.0
    
def quip_B(x):
    if math.isnan(x['CNTRLSEX']):
        return x['CNTRLSEX']
    if math.isnan(x['TMSEX']):
        return x['TMSEX']
    if x['CNTRLSEX']==1.0 or x['TMSEX']==1.0:
        return 1.0
    else:
        return 0.0
    
def quip_C(x):
    if math.isnan(x['CNTRLBUY']):
        return x['CNTRLBUY']
    if math.isnan(x['TMBUY']):
        return x['TMBUY']
    if x['CNTRLBUY']==1.0 or x['TMBUY']==1.0:
        return 1.0
    else:
        return 0.0
    
def quip_D(x):
    if math.isnan(x['CNTRLEAT']):
        return x['CNTRLEAT']
    if math.isnan(x['TMEAT']):
        return x['TMEAT']
    if x['CNTRLEAT']==1.0 or x['TMEAT']==1.0:
        return 1.0
    else:
        return 0.0

def quip_E(x):
    sum=0
    for col in ["TMTORACT","TMTMTACT","TMTRWD"]:
        if math.isnan(x[col]):
            return x[col]
        if x[col]==1.0:
            sum+=1.0
    return sum

In [ ]:
df_8['QUIP_A'] = df_8.apply(quip_A,axis=1)
df_8.head(3)

In [ ]:
df_8['QUIP_B'] = df_8.apply(quip_B,axis=1)
df_8['QUIP_C'] = df_8.apply(quip_C,axis=1)
df_8['QUIP_D'] = df_8.apply(quip_D,axis=1)
df_8['QUIP_E'] = df_8.apply(quip_E,axis=1)
df_8.head(5)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_8[['PATNO','EVENT_ID','INFODT','QUIP_A','QUIP_B','QUIP_C','QUIP_D','QUIP_E']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(3)

In [ ]:
# Processing the feature for df_7 - Olfactory UPSIT
# We will actually load the DF since df_7 has the more granular results
file = os.path.join(path, "University_of_Pennsylvania_Smell_ID_Test.csv")
df_upsit = pd.read_csv(file)

# Guidelines
# Sum of UPSITBK1 - UPSITBK4

In [ ]:
# df_upsit.head(3)

In [ ]:
df_upsit['UPSIT_TOT'] = df_upsit[['UPSITBK1','UPSITBK2','UPSITBK3','UPSITBK4']].sum(axis=1)
df_upsit.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_upsit[['PATNO','EVENT_ID','INFODT','UPSIT_TOT']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(3)

In [ ]:
# Processing the feature for df_6 - Montreal Cognitive Assessment (MoCA)

# Guidelines
# Unadjusted Score = sum of MCAALTTM, MCACUBE, MCACLCKC, MCACLCKN, MCACLCKH, 
# MCALION,  MCARHINO, MCACAMEL, MCAFDS, MCABDS, MCAVIGIL, MCASER7, MCASNTNC, 
# MCAVF, MCAABSTR, MCAREC1, MCAREC2, MCAREC3, MCAREC4, MCAREC5, MCADATE, MCAMONTH, 
# MCAYR, MCADAY, MCAPLACE, MCACTY.  If EDUCYRS <=12 
# and Unadjusted Score < 30, add 1 more point to score.  
# If EDUCYRS > 12, do not add any more points to score.

In [ ]:
df_6.head(3)

In [ ]:
# Loading the socio-eco table in
file = os.path.join(path, "Socio-Economics.csv")
df_se = pd.read_csv(file)
df_se.head(3)

In [ ]:
df_6_se = pd.merge(left=df_6, right=df_se[['PATNO','EDUCYRS']], \
                        on=['PATNO'])
df_6_se.head(3)

In [ ]:
df_6_se['Unadjusted_TOT'] = df_6_se[["MCAALTTM","MCACUBE","MCACLCKC","MCACLCKN",\
                                     "MCACLCKH","MCALION","MCARHINO","MCACAMEL",\
                                     "MCAFDS","MCABDS","MCAVIGIL","MCASER7","MCASNTNC",\
                                     "MCAVF","MCAABSTR","MCAREC1","MCAREC2","MCAREC3",\
                                     "MCAREC4","MCAREC5","MCADATE","MCAMONTH","MCAYR",\
                                     "MCADAY","MCAPLACE","MCACITY"]].sum(axis=1)

In [ ]:
df_6_se.head(3)

In [ ]:
df_6_se.Unadjusted_TOT.isnull().sum()

In [ ]:
a=np.where(((df_6_se['Unadjusted_TOT']<30) & (df_6_se['EDUCYRS']<=12)), df_6_se['Unadjusted_TOT']+1, df_6_se['Unadjusted_TOT'])

In [ ]:
print(len(a),len(df_6_se))

In [ ]:
df_6_se['MoCA_score'] = a
df_6_se.head(3)

In [ ]:
df_6_se[((df_6_se['Unadjusted_TOT']<30) & (df_6_se['EDUCYRS']<=12))][:3]

In [ ]:
df_6_se.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_6_se[['PATNO','EVENT_ID','INFODT','MoCA_score']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(5)

In [ ]:
# Processing the feature for df_5 - Letter Number Sequencing (LNSPD)

# Guidelines
# Sum of LNS1A - LNS7C
df_5.head(3)

In [ ]:
df_5['LNS_TOT'] = df_5[["LNS1A","LNS1B","LNS1C","LNS2A","LNS2B","LNS2C","LNS3A","LNS3B",
       "LNS3C","LNS4A","LNS4B","LNS4C","LNS5A","LNS5B","LNS5C","LNS6A",
       "LNS6B","LNS6C","LNS7A","LNS7B","LNS7C"]].sum(axis=1)

In [ ]:
df_5.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_5[['PATNO','EVENT_ID','INFODT','LNS_TOT']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(5)

In [ ]:
# Processing the feature for df_4 - Hopkins Verbal Learning Test – Revised (HVLT)

# Guidelines
# HVLT Immediate/Total Recall	Sum of HVLTRT1 - HVLTRT3
# HVLT Discrimination Recognition	HVLTREC - (HVLTFPRL + HVLTFPUN)
# HVLT Retention	HVLTRDLY / max(HVLTRT2, HVLTRT3)
df_4.head(3)

In [ ]:
df_4['HVLT_TOT_Recall'] = df_4[["HVLTRT1","HVLTRT2","HVLTRT3"]].sum(axis=1)

In [ ]:
df_4['HVLT_DCR_REC'] = df_4['HVLTREC'] - df_4['HVLTFPRL'] - df_4['HVLTFPUN']
df_4.head(3)

In [ ]:
df_4['HVLTREC'].isnull().sum()

In [ ]:
df_4['HVLT_DCR_REC'].isnull().sum()

In [ ]:
len(df_4)

In [ ]:
# Defining functions to apply for the other two columns in aggregation
def hvlt_retention(x):
    if math.isnan(x['HVLTRDLY']):
        return x['HVLTRDLY']
    if math.isnan(x['HVLTRT2']):
        return x['HVLTRT2']
    if math.isnan(x['HVLTRT3']):
        return x['HVLTRT3']
    return x['HVLTRDLY']/max(x['HVLTRT2'],x['HVLTRT3'])

In [ ]:
df_4['HVLT_RETENTION'] = df_4.apply(hvlt_retention,axis=1)
df_4.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_4[['PATNO','EVENT_ID','INFODT','HVLT_TOT_Recall','HVLT_DCR_REC','HVLT_RETENTION']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(5)

In [ ]:
# Processing the feature for df_3 - Geriatric Depression Scale (GDS-15)

# Guidelines
# Add 1 point for each response of "No" (0) to any of the following variables:  
# GDSSATIS, GDSGSPIR, GDSHAPPY, GDSALIVE, GDSENRGY. 

# Add 1 point for each response of "Yes" (1) to any of the following variables:  
# GDSDROPD, GDSEMPTY, GDSBORED, GDSAFRAD, GDSHLPLS, GDSHOME, GDSMEMRY, GDSWRTLS, 
# GDSHOPLS, GDSBETER.  

# Subjects with GDS >=5 are "Depressed".  Subjects with GDS <5 are "Not Depressed".
df_3.head(3)

In [ ]:
df_3['GDS_NO_Count'] = 5.0 - df_3[["GDSSATIS","GDSGSPIR","GDSHAPPY","GDSALIVE","GDSENRGY"]].sum(axis=1)
df_3.head(3)

In [ ]:
df_3['GDS_YES_Count'] = df_3[["GDSDROPD","GDSEMPTY","GDSBORED","GDSAFRAD","GDSHLPLS","GDSHOME",
                              "GDSMEMRY","GDSWRTLS","GDSHOPLS","GDSBETER"]].sum(axis=1)
df_3['GDS_TOT'] = df_3[['GDS_NO_Count','GDS_YES_Count']].sum(axis=1)
df_3.head(3)

In [ ]:
df_3['GDS_Depressed'] = np.where((df_3['GDS_TOT']>=5), 1.0, 0.0) 
df_3.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_3[['PATNO','EVENT_ID','INFODT','GDS_TOT','GDS_Depressed']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(5)

In [ ]:
# Processing the feature for df_2 - Epworth Sleepiness Scale (ESS)

# Guidelines
# Sum of ESS1 - ESS8.  
# Subjects with ESS <10 are "Not Sleepy".  
# Subjects with ESS >=10 are "Sleepy".
df_2.head(3)

In [ ]:
# 1 means sleepy and 0 means not sleepy
df_2['ESS_TOT'] = df_2[["ESS1","ESS2","ESS3","ESS4","ESS5","ESS6","ESS7","ESS8"]].sum(axis=1)
df_2['ESS_Sleepy'] = np.where((df_2['ESS_TOT']>=10), 1.0, 0.0)
df_2.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_2[['PATNO','EVENT_ID','INFODT','ESS_TOT','ESS_Sleepy']], \
                        how='outer', on=['PATNO','EVENT_ID','INFODT'])
df_processed.head(5)

In [ ]:
# Processing the feature for df_1 - Benton Judgement of Line Orientation (BJLO)

# Guidelines
# Sum of BJLOT1 - BJLOT30
df_1.head(3)

In [ ]:
def create_list(text):
    res=[]
    for i in range(30):
        res.append(text+str(i+1))
    return res

In [ ]:
print(create_list('BJLOT'))

In [ ]:
df_1['BJLOT_TOT'] = df_1[['BJLOT1', 'BJLOT2', 'BJLOT3', 'BJLOT4', 'BJLOT5', 'BJLOT6', 'BJLOT7', 
                        'BJLOT8', 'BJLOT9', 'BJLOT10', 'BJLOT11', 'BJLOT12', 'BJLOT13', 
                        'BJLOT14', 'BJLOT15', 'BJLOT16', 'BJLOT17', 'BJLOT18', 'BJLOT19', 
                        'BJLOT20', 'BJLOT21', 'BJLOT22', 'BJLOT23', 'BJLOT24', 'BJLOT25', 
                        'BJLOT26', 'BJLOT27', 'BJLOT28', 'BJLOT29', 'BJLOT30']].sum(axis=1)
df_1.head(3)

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_1[['PATNO', 'EVENT_ID', 'INFODT', 'BJLOT_TOT']], 
                         how='outer', on=['PATNO','EVENT_ID','INFODT'])

In [ ]:
df_datscn = pd.read_csv(path + 'DATSCAN_Analysis.csv')
df_datscn.head()

In [ ]:
df_processed = pd.merge(left=df_processed, right=df_datscn, how='outer', on=['PATNO', 'EVENT_ID'])

## We are all done here!!! :)

In [ ]:
df_processed.to_csv("non-motor.csv")